In [ ]:
!pip install -U pip wheel setuptools

In [ ]:
!pip install concrete-ml

In [123]:
import time
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor as SklearnMLPRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import NeuralNetRegressor as ConcreteNNRegressor
import torch
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd
#!unzip house.zip

In [194]:
house_dataset = pd.read_csv("./Housing.csv")
df = pd.DataFrame(house_dataset)
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [196]:
#preprocessing

scaler_cat = OneHotEncoder()
scaler_y = MinMaxScaler()
std = np.std(df['price'])
y = np.array(df.pop('price'), dtype=np.float32)
y = np.expand_dims(y, axis=-1)
y = scaler_y.fit_transform(y)
x = np.empty((len(df), 0))

categorical_features = df.select_dtypes(include=['object']).columns.tolist()

for key, values in df.items():
  el = np.expand_dims(np.array(df[key]), axis=-1)
  if key in categorical_features:
    el = scaler_cat.fit_transform(el).toarray()
  x = np.concatenate((x, el), axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [199]:
#sklearn small model

sklearn_MLP_regressor = SklearnMLPRegressor(
                    alpha=1,
                    activation="logistic",
                    max_iter=1000,
                    hidden_layer_sizes=(25,),
                    learning_rate_init=0.005,
                    random_state = 42)
sklearn_MLP_regressor.fit(x_train, y_train)

y_pred = sklearn_MLP_regressor.predict(x_train)
y_pred = scaler_y.inverse_transform(np.expand_dims(y_pred, axis=-1))
y_true = scaler_y.inverse_transform(y_train)
msa = mean_absolute_error(y_true, y_pred)
print(f'msa on train data: {round(msa/1000000, 2)} millions, ie {round(msa/std, 3)} std')

execution_time_on_plaintext = 0
for i in range(10010):
  time_begin = time.time()
  y_pred = sklearn_MLP_regressor.predict(x_test)
  time_end = time.time()
  if i > 10:
    execution_time_on_plaintext += (time_end - time_begin) / len(x_test)
execution_time_on_plaintext /= 10000
print(f"Model evaluation time on the clear: {execution_time_on_plaintext*1000000:.2f} microseconds per sample")

y_pred = sklearn_MLP_regressor.predict(x_test)
y_pred = scaler_y.inverse_transform(np.expand_dims(y_pred, axis=-1))
y_true = scaler_y.inverse_transform(y_test)
msa = mean_absolute_error(y_true, y_pred)
print(f'msa on test data: {round(msa/1000000, 2)} millions, ie {round(msa/std, 3)} std')

msa train: 1.33 millions, ie 0.713 std
Model evaluation time on the clear: 2.40 microseconds per sample
msa test: 1.75 millions, ie 0.935 std


In [204]:
# Instantiate the model with parameter
# TODO: tune the parameter
params_neural_net = {
    # "module__n_w_bits": 6,
    # "module__n_a_bits": 8,
    # "module__n_accum_bits": 16,
    "module__n_hidden_neurons_multiplier": 5,
    "module__n_layers": 1,  # total number of layers in the FCNN = 1 hidden layer
    "module__activation_function": torch.nn.ReLU,
    "max_epochs": 10,
    "verbose": 1,
    # "lr": 0.1,
}


#some sort of Feature preprocessing needed for quantization aware training
# Linear models require polynomial features to be applied before training to fit a non-linear model and other models perform better with this transoformation
pipe = Pipeline(
    [
        ("poly", PolynomialFeatures()),
        ("scaler", StandardScaler()),
    ]
)

X_poly_train = pipe.fit_transform(x_train)
X_poly_test = pipe.transform(x_test)

concrete_NN_regressor = ConcreteNNRegressor(batch_size=32, **params_neural_net)

# train the concrete linear regression model on clear data
# The built-in NN regressor models will automatically quantize weights and activations with .fit() call. (Quantization Aware Training) These models use several layers for Quantization Aware Training, allowing good performance for low precision (down to 2-3 bits) weights and activations.
# The maximum accumulator bit-width is controlled by the number of weights and activation bits, as well as a pruning factor. This factor is automatically determined based on the desired accumulator bit-width and a multiplier factor can be optionally specified.


concrete_NN_regressor.fit(X_poly_train, y_train)
print("done training")

y_pred_q = concrete_NN_regressor.predict(X_poly_train)
y_pred_q = scaler_y.inverse_transform(y_pred_q)
y_true = scaler_y.inverse_transform(y_train)
msa = mean_absolute_error(y_true, y_pred_q)
print(f'msa train: {round(msa/1000000, 2)} million $, ie {round(msa/std, 3)} std')


y_pred_q = concrete_NN_regressor.predict(X_poly_test)
y_pred_q = scaler_y.inverse_transform(y_pred_q)
y_true = scaler_y.inverse_transform(y_test)
msa = mean_absolute_error(y_true, y_pred_q)
print(f'msa test: {round(msa/1000000, 2)} million $, ie {round(msa/std, 3)} std')


  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.3707        0.2650  0.1121
      2        0.3074        0.3057  0.1248
      3        0.1932        0.2641  0.1758
      4        0.1711        0.1761  0.1036
      5        0.1122        0.0548  0.1368
      6        0.0878        0.0727  0.1438
      7        0.0679        0.0522  0.1039
      8        0.0517        0.1512  0.1161
      9        0.0758        0.1002  0.0626
     10        0.0499        0.0682  0.0682
done training
msa train: 2.06 millions, ie 1.101 std
msa test: 2.06 millions, ie 1.102 std


In [208]:
# Compile the quantized model in to FHE circuit and run inference on it
# You have to provide the training dataset in order to compile the quantized model to equivalent FHE circuit
time_begin = time.time()
fhe_circuit = concrete_NN_regressor.compile(X_poly_train)
print(f"Generating a key for a {fhe_circuit.graph.maximum_integer_bit_width()}-bit circuit")
print(f"Compilation time: {time.time() - time_begin:.4f} seconds")

# Compiler returns the circuit, which can be used to generated a secrete key and evaluation key
# secrete key: used for encryption and decryption. only accesible to the client
# evaluation key: used to evaluate the cirucit on encypted data. anyone can access it
print(f"Generating a key for a {fhe_circuit.graph.maximum_integer_bit_width()}-bit circuit")

time_begin = time.time()
fhe_circuit.client.keygen(force=False)
print(f"Key generation time: {time.time() - time_begin:.4f} seconds")

Generating a key for a 8-bit circuit
Compilation time: 0.4413 seconds
Generating a key for a 8-bit circuit
Key generation time: 0.0006 seconds


In [217]:
# Evaluate the FHE-version of the model
time_begin = time.time()
y_pred_fhe = concrete_NN_regressor.predict(X_poly_test, fhe="execute")

execution_time_on_ciphertext = (time.time() - time_begin) / len(x_test)
print(f"Execution time: {execution_time_on_ciphertext*1000:.2f} milliseconds per sample")
print(f"which is {(execution_time_on_ciphertext / execution_time_on_plaintext):.0f} times slower than prediction on the plaintext data")

Execution time: 20.03 milliseconds per sample
which is 8353 times slower than prediction on the plaintext data


In [218]:
y_pred_fhe = scaler_y.inverse_transform(y_pred_fhe)
y_true = scaler_y.inverse_transform(y_test)
msa = mean_absolute_error(y_true, y_pred_fhe)
print(f'msa test: {round(msa/1000000, 2)} million $, ie {round(msa/std, 3)} std')

msa test: 2.06 million $, ie 1.102 std
